In [ ]:
import cv2
import dlib
import numpy as np
import os
from imutils import face_utils
import pandas as pd


In [8]:

# Load the pre-trained face detector and landmark predictor from dlib
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Constants for blink detection
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3

# detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# print("Dlib modules loaded successfully")

In [9]:
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])

    # Compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = np.linalg.norm(eye[0] - eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear


In [15]:
def process_video(video_path, duration=30):
    cap = cv2.VideoCapture(video_path)
    blink_count = 0
    total_eye_closure_duration = 0
    frame_count = 0
    open_eyes_durations = []
    closed_eyes_durations = []
    consecutive_closed_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[face_utils.FACIAL_LANDMARKS_IDXS["left_eye"][0]:face_utils.FACIAL_LANDMARKS_IDXS["left_eye"][1]]
            rightEye = shape[face_utils.FACIAL_LANDMARKS_IDXS["right_eye"][0]:face_utils.FACIAL_LANDMARKS_IDXS["right_eye"][1]]
            
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR + rightEAR) / 2.0

            if ear < EYE_AR_THRESH:
                consecutive_closed_frames += 1
            else:
                if consecutive_closed_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1
                    closed_eyes_durations.append(consecutive_closed_frames)
                consecutive_closed_frames = 0
                open_eyes_durations.append(frame_count)

        frame_count += 1
        if frame_count >= duration * cap.get(cv2.CAP_PROP_FPS):
            break

    cap.release()
    cv2.destroyAllWindows()

    open_eye_avg_duration = np.mean(open_eyes_durations) if open_eyes_durations else 0
    closed_eye_avg_duration = np.mean(closed_eyes_durations) if closed_eyes_durations else 0
    eye_closure_ratio = sum(closed_eyes_durations) / frame_count if frame_count > 0 else 0

    return {
        'blink_count': blink_count,
        'closed_eye_avg_duration': closed_eye_avg_duration,
        'open_eye_avg_duration': open_eye_avg_duration,
        'eye_closure_ratio': eye_closure_ratio
    }

def process_videos_in_folder(folder_path, label, duration=30):
    results = []
    i = 0
    for filename in os.listdir(folder_path):	
        if filename.endswith(".mp4") and i < 10:
            video_path = os.path.join(folder_path, filename)
            i = i+1
            # print(video_path)
            features = process_video(video_path, duration)
            features['label'] = label
            results.append(features)
    return results

# Process videos in 'data/drowsiness' folder
drowsiness_folder = 'data/drowsiness'
drowsiness_results = process_videos_in_folder(drowsiness_folder, label=1)

# Process videos in 'data/not_drowsiness' folder
not_drowsiness_folder = 'data/not drowsiness'
not_drowsiness_results = process_videos_in_folder(not_drowsiness_folder, label=0)


In [16]:
drowsiness_results

[{'blink_count': 2,
  'closed_eye_avg_duration': np.float64(93.0),
  'open_eye_avg_duration': np.float64(136.24137931034483),
  'eye_closure_ratio': 0.62,
  'label': 1},
 {'blink_count': 3,
  'closed_eye_avg_duration': np.float64(32.0),
  'open_eye_avg_duration': np.float64(113.4),
  'eye_closure_ratio': 0.32,
  'label': 1},
 {'blink_count': 3,
  'closed_eye_avg_duration': np.float64(9.0),
  'open_eye_avg_duration': np.float64(126.91928251121077),
  'eye_closure_ratio': 0.108,
  'label': 1},
 {'blink_count': 3,
  'closed_eye_avg_duration': np.float64(5.666666666666667),
  'open_eye_avg_duration': np.float64(120.13304721030043),
  'eye_closure_ratio': 0.068,
  'label': 1},
 {'blink_count': 2,
  'closed_eye_avg_duration': np.float64(7.5),
  'open_eye_avg_duration': np.float64(121.7008547008547),
  'eye_closure_ratio': 0.06,
  'label': 1},
 {'blink_count': 2,
  'closed_eye_avg_duration': np.float64(12.0),
  'open_eye_avg_duration': np.float64(122.39823008849558),
  'eye_closure_ratio': 0.